In [29]:
#!pip install opencv-python

In [30]:
import cv2 
import sys #using for face recognition
import os
from pathlib import Path
import glob
import random
from skimage.util import random_noise ## --> to add noise
import shutil
import numpy as np
import pandas as pd

In [46]:
test_set = pd.read_csv('./Celeb-DF-v2/List_of_testing_videos.txt',sep=' ')
test_set.columns = ['label','filepath']

In [47]:
test_set.head()

,label,filepath
0,1,YouTube-real/00208.mp4
1,1,YouTube-real/00063.mp4
2,1,YouTube-real/00024.mp4
3,1,YouTube-real/00021.mp4
4,1,YouTube-real/00036.mp4


In [48]:
test_set.label.value_counts()

0    340
1    177
Name: label, dtype: int64

In [49]:
import os,shutil

directory = './Celeb-DF-v2'
new_directory = './Celeb-DF-v2/test'

names = []

for filename in test_set['filepath']:
    names.append(filename)
#     new_name = filename.split('/')[-2]+'-'+filename.split('/')[-1]
#     #print(new_name)
    
#     filepath = os.path.join(directory,filename)
#     print(filepath)
    
#     shutil.copy(filepath, new_directory)
#     file = os.path.join(new_directory,filename.split('/')[-1])
#     #print(file)
#     os.rename(file, new_name)
    
    #print(new_name)

#shutil.copy(filepath,new_directory)

In [50]:
test_set.filepath = [file.split('/')[-2]+'-'+file.split('/')[-1] for file in test_set.filepath]

In [51]:
test_set.head()

,label,filepath
0,1,YouTube-real-00208.mp4
1,1,YouTube-real-00063.mp4
2,1,YouTube-real-00024.mp4
3,1,YouTube-real-00021.mp4
4,1,YouTube-real-00036.mp4


In [52]:
test_set.label.value_counts()

0    340
1    177
Name: label, dtype: int64

In [59]:
for index, row in test_set.iterrows():
    if row[0] == 0:
        test_set.label[index] = 1
    else:
        test_set.label[index] = 0

<ipython-input-59-e507acee01c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.label[index] = 0
<ipython-input-59-e507acee01c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.label[index] = 1


In [60]:
test_set.label.value_counts()

1    340
0    177
Name: label, dtype: int64

In [61]:
test_set.head()

,label,filepath
0,0,YouTube-real-00208.mp4
1,0,YouTube-real-00063.mp4
2,0,YouTube-real-00024.mp4
3,0,YouTube-real-00021.mp4
4,0,YouTube-real-00036.mp4


In [66]:
'''
func: captures frames from the video, adds noise to the video and saves in the given folder
input:
    i.vid_path: string: the name of the video 
    ii.target: string: the file path for the frames folder (Eg: ./Dataset/FaceSwap/frames)
    iii. req_fps: int: required video fps for frame extraction. No.of frames extracted will be based on this
    iv. faceCrop: bool: value to determine if face cropping is necessary or no
'''
def FrameCapture(vid_path,target): 
    # Path to video file 
    vidObj = cv2.VideoCapture(vid_path)
    vidObj.set(cv2.CAP_PROP_POS_MSEC,2000)
    # Used as counter variable 
    # checks whether frames were extracted 
    
    path = Path(vid_path)
    video = path.parts[-1]
    video = video.split('.')[0]
    
    success, image = vidObj.read() 

    if success: 
        # vidObj object calls read 
        if np.shape(image) != ():
            #print('Empty Image Error')
            
            framepath = os.path.join('./Celeb-DF-v2/test/frames/'+video+".jpg")
            img = AddNoise(image)
            crop_img = CropFace(img)
            try:
                print('Write ',framepath)
                cv2.imwrite(framepath, crop_img)
            except Exception as e:
                print(e)

In [67]:
'''
func: add gaussian noise to the frames 
input:
    i.image: object: image object video frame
'''
def AddNoise(image):
    # Add gaussian noise to the image.
    noise_img = random_noise(image, mode='gaussian')

    # The above function returns a floating-point image
    # on the range [0, 1], thus we changed it to 'uint8'
    # and from [0,255]
    noise_img = np.array(255*noise_img, dtype = 'uint8')
    return noise_img
    #cv2.imwrite(image_path, noise_img)

In [68]:
'''
func: locate face using CV2 and crop it. 
input:
    i.image: object: image object video frame
'''
def CropFace(image):
    
    #reading the image
    #image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #finding a face
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
            
    #cropping the image
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = image[y-20:y + h + 20, x-20:x + w + 20] #the + and - 20 gives a little bit of border around the face
        return roi_color

In [69]:
'''
func: main pre-processing function 
input:
    i.vid_dir: string: video filepath
    ii. extension: string: 2 possible values frames or face_crop
    iii. faceCrop: bool: boolean value to determine if face cropping is needed
'''
#Main Pre-Process Function
def Pre_Process(vid_dir, extension):
    for video in vid_dir:
        files = glob.glob('./Celeb-DF-v2/test/*.mp4', recursive = True) 
        target = os.path.join(video+'.mp4')
        for file in files:
            FrameCapture(file,target)
        print('Pre-Process Complete')

In [70]:
Pre_Process('./Celeb-DF-v2/test/','.mp4')

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id53_0000.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00257.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id21_id19_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id3_id1_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id7_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id35_id31_0007.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00280.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id4_id0_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id44_id42_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id39_0004.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00082.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id3_id28_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id4_id23_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id22_id27_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id9_id3_0009.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_i

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id54_id49_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_id35_0006.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00285.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id11_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id4_id0_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id25_0010.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id28_id21_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id28_id20_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id33_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id32_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id40_0009.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id10_id7_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id37_id3_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id45_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id36_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id44_0003.jpg
Write  .

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id17_id0_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id3_id20_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id17_id1_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id42_id40_0001.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00193.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id30_id4_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id34_0006.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id17_id28_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id20_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id33_id32_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id35_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id37_id4_0000.jpg
Wri

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id50_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id3_id2_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id19_id20_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id35_id32_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id39_id47_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id21_id4_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id9_id26_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id43_id40_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id22_id25_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id30_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id31_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id37_id21_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id29_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id30_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id32_id33_0002.jpg
Write  ./Celeb-DF-v2/tes

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id32_0008.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id36_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id25_id28_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id23_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id16_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_id3_0009.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id27_id19_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id51_id50_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id13_id7_0012.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id35_id16_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id37_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id2_id0_0008.jpg
W

...

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id36_0002.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id25_id28_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id23_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id16_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_id3_0009.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id27_id19_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id51_id50_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id13_id7_0012.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id35_id16_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id37_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id2_id0_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id3_0002.jpg
Wr

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id30_id3_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id4_id28_0006.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id4_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id9_id6_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_id3_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id21_id9_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id30_id28_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id32_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id35_id26_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id0_id16_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id33_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id28_id4_0000

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id28_0004.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id3_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id35_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id1_id3_0003.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00194.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id43_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id28_id35_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id43_id39_0000.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id38_id30_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id1_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id54_0007.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id0_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id2_id9_0008.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00023.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id16_0010.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id55_id57_0002.jpg
Write  ./Celeb

Write  ./Celeb-DF-v2/test/frames/Celeb-real-id35_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id10_id12_0001.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id10_id13_0001.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id28_id6_0006.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id31_0003.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id44_id41_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id13_0013.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-real-id1_0008.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id19_id20_0004.jpg
Write  ./Cel

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id16_id2_0011.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00263.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00275.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00261.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id6_id16_0005.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id29_id38_0008.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id31_id16_0002.jpg
OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-2rx9f0ng/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id21_id2_0002.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id23_id35_0009.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id20_id31_0006.jpg
Write  ./Celeb-DF-v2/test/frames/YouTube-real-00063.jpg
Write  ./Celeb-DF-v2/test/frames/Celeb-synthesis-id3_id4_0001.jpg
Write  ./Celeb-DF-v2/test/fr